In [1]:
import sys
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.utils import Sequence
from keras.utils import load_img

2023-01-08 22:52:05.860122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 22:52:06.064356: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /appl/cuda/12.0.0/lib64:/appl/cudnn/v8.3.2.44-prod-cuda-11.5/lib:/appl/python/3.10.7/lib:/appl/gcc/11.3.0-binutils-2.38/lib64:/appl/gcc/11.3.0-binutils-2.38/lib:/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2023-01-08 22:52:06.064391: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-08 22:52

In [2]:
w,h = 800,600

In [3]:
df = pd.read_csv('lables.csv')
train_df = df[df['ImagePath'].str.contains("train")]
test_df = df[df['ImagePath'].str.contains("test")]
valid_df = df[df['ImagePath'].str.contains("valid")]

In [4]:
#########################################################
################## data generator #######################
#########################################################
class datagenerator(tf.keras.utils.Sequence):
    def __init__(self, 
            batch_size, 
            img_size,
            data_paths_df,
            input_channels,
            output_channels):
         
        self.batch_size = batch_size
        self.img_size = img_size
        self.data_paths_df = data_paths_df
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.data_paths = data_paths_df.values[:,1]
        self.params = data_paths_df.values[:,3:6]
        assert len(self.data_paths) == len(self.params)
        
        self.n = len(self.data_paths)

    def on_epoch_end(self):
        'updates indexes after each epoch'
        self.data_paths_df = self.data_paths_df.sample(frac = 1)
        self.data_paths = self.data_paths_df.values[:,1]
        self.params = self.data_paths_df.values[:,3:6]
    
    def __getitem__(self, index):
        batch_data_paths = self.data_paths[index : index + self.batch_size]
        batch_params_paths = self.params[index : index + self.batch_size]

        return self.__dataloader(self.img_size,
                batch_data_paths, batch_params_paths,
                self.input_channels, self.output_channels)
    
    def __len__(self):
        return self.n // self.batch_size

    #################### data loader ########################
    def __dataloader(self, 
            img_size,
            data_paths,
            batch_params_paths,
            input_channels,
            output_channels):
        x = np.zeros((len(data_paths), img_size[0], img_size[1], input_channels))
        y = batch_params_paths
        
        
        for i in range(len(data_paths)):
            data = load_img(path = data_paths[i], grayscale = True)
            data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
            data /= 255
            data.shape = (1,) + data.shape
            x[i] = np.asarray(data)
        return np.array(x).astype("float32"), np.array(y).astype("float32")

In [5]:
inputs = keras.Input(shape=(600, 800, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(3, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)
model_no_max_pool.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 600, 800, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 598, 798, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 299, 399, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 297, 397, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 148, 198, 64)     0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1875456)           0     

2023-01-08 22:52:10.725451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /appl/cuda/12.0.0/lib64:/appl/cudnn/v8.3.2.44-prod-cuda-11.5/lib:/appl/python/3.10.7/lib:/appl/gcc/11.3.0-binutils-2.38/lib64:/appl/gcc/11.3.0-binutils-2.38/lib:/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2023-01-08 22:52:10.726446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /appl/cuda/12.0.0/lib64:/appl/cudnn/v8.3.2.44-prod-cuda-11.5/lib:/appl/python/3.10.7/lib:/appl/gcc/11.3.0-binutils-2.38/lib64:/appl/gcc/11.3.0-binutils-2.38/lib:/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2023-01-08 22:52:10.727178: W tensorflow/compiler/xla/stream_executor/platform

In [6]:
def custom_loss_function(y_true, y_pred):
    # Calculate the error for each parameter using sqrt(2 - cos(y_hat - y_pred))
    param_1_error = tf.math.sqrt(2 - tf.math.cos(y_true[:, 0] - y_pred[:, 0]))
    param_2_error = tf.math.sqrt(2 - tf.math.cos(y_true[:, 1] - y_pred[:, 1]))
    param_3_error = tf.math.sqrt(2 - tf.math.cos(y_true[:, 2] - y_pred[:, 2]))

    # Calculate the total loss as the mean of the errors for each parameter
    total_loss = (param_1_error + param_2_error + param_3_error) / 3
    
    return total_loss

model_no_max_pool.compile(optimizer='adam', loss=custom_loss_function)
tg = datagenerator(32, (600,800), train_df, 1, 3)
vg = datagenerator(32, (600,800), valid_df, 1, 3)

In [7]:
history = model_no_max_pool.fit(x=tg,
                    batch_size=32,
                    epochs=5,
                    validation_data=vg)

/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/keras/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


Epoch 1/5


2023-01-08 22:52:13.102013: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1954627584 exceeds 10% of free system memory.
2023-01-08 22:52:13.445886: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 965910528 exceeds 10% of free system memory.
2023-01-08 22:52:13.705603: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 965910528 exceeds 10% of free system memory.
2023-01-08 22:52:15.762157: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 977313792 exceeds 10% of free system memory.
2023-01-08 22:52:15.762384: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1954627584 exceeds 10% of free system memory.


  4/187 [..............................] - ETA: 13:21 - loss: 1.4048

KeyboardInterrupt: 

In [ ]:
t = test_df.values[4][1]
data = load_img(path = t, grayscale = True)
data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
data /= 255
data.shape = (1,) + data.shape
X = np.asarray(data)

yhat = model_no_max_pool.predict(data)
print(t)
print(yhat*90)